In [2]:
import random, math

TODO: Get rid of this ghetto copy paste import and actually import it and put it in the github repository

In [199]:
import json
class NodeAlreadyExistsError(Exception):
    pass
class DiGraph(dict):
    def add_node(self,*nodes):
        '''Add a node to the graph. Can be of any hashable type.
        Can add multiple nodes like G.add_node(n1,n2,...).
        Raises error if node exists'''
        for node in nodes:
            if node in self.keys():
                raise NodeAlreadyExistsError(node)
            else:
                self[node] = {'to':{},'from':{}}
    def set_edge(self,a,b,obj={}):
        '''Create an edge between nodes a and b.
        Updates/overwrites edge if one exists.'''
        if isinstance(obj, (int, float)):
            obj = {'weight':obj}
        elif isinstance(obj,str):
            obj = {'rel':obj}
        assert isinstance(obj,dict)
        try:
            #update edge
            edgeobj = {**self[a][b],**obj}
            self[a]['to'][b] = edgeobj
            self[b]['from'][a] = self[a]['to'][b]
        except KeyError as e:
            #no edge yet
            self[a]['to'][b] = obj
            self[b]['from'][a] = self[a]['to'][b]
    def remove_edge(self,a,b):
        # print(self,a,b)
        try:
            del self[a]['to'][b]
        except KeyError:
            pass
        try:
            del self[b]['from'][a]
        except KeyError:
            pass
    def remove_node(self,node):
        #remove children edges, parent edges, and del node
        for child in self.get_children(node):
            self.remove_edge(node,child)
        for parent in self.get_parents(node):
            self.remove_edge(parent,node)
        del self[node]
    def get_nodes(self):
        '''returns the set of nodes'''
        return set(self.keys())
    def get_children(self,node):
        '''returns a tuple containing children of node'''
        return tuple(self[node]['to'].keys())
    def get_parents(self,node):
        '''Returns a tuple containing parents of node'''
        return tuple(self[node]['from'].keys())
    def pretty(self):
        '''returns json string with indentation'''
        return json.dumps(self, sort_keys=True, indent=2)
    def get_edges(self):
        ans = set([])
        for a in self:
            for b in self[a]['to']:
                ans.add((a,b))
        return ans

chooses a random element, taking into account weightings

In [200]:
def randChoiceWithWeights(probs):
    '''expects probs to be {key:prob,key:prob,...}'''
    choices = []#[(num, key),(num,key),...]
    total = 0
    for key in probs:
        choices.append((total,key))
        total += probs[key]
    num = random.random()*total
    ind = 0
#     print(choices,num)
    for choice in choices[::-1]:
        if choice[0] < num:
            return choice[1]
    return choices[0][1]
d = {'a':.1,'b':.7,'c':.2}
randChoiceWithWeights(d)

'c'

The actual class (extends a directional graph)

In [201]:
class Markov(DiGraph):
    def add_edge(self,a,b,prob):
        if not isinstance(prob, (int, float)):
            raise TypeError('edge must be a number')
        else:
            assert prob >= 0
            return super().add_edge(self,a,b,prob)
    def start(self,node):
        assert node in self.get_nodes()
        self.current = node
    def next(self):
        assert self.current in self.get_nodes()
        probs = {}
        for child in  self.get_children(self.current):
            probs[child] = self[self.current]['to'][child]['weight']
#         print(probs)
        ans = randChoiceWithWeights(probs)
        self.current = ans
        return randChoiceWithWeights(probs)
    def from_adj(adj,names=[]):
        '''adj is by row to by col
        ex: 
          0 1
      0 [[0,.5],
      1  [0,0]]
        is node 0 to node 1  with prob .5 and no other edges'''
        G = Markov()
        if not names:
            names = list(range(len(adj)))
        assert len(names) == len(adj)
        G.add_node(*names)
        for i in range(len(adj)):
            for j in range(len(adj)):
                G.set_edge(names[i],names[j],adj[i][j])
        return G

Normal instantiation

In [205]:
G = Markov()
G.add_node('rain','clear')
G.set_edge('rain','rain',.9)
G.set_edge('clear','clear',.9)
G.set_edge('rain','clear',.1)
G.set_edge('clear','rain',.1)
G.start('rain')
print(G.pretty())

{
  "clear": {
    "from": {
      "clear": {
        "weight": 0.9
      },
      "rain": {
        "weight": 0.1
      }
    },
    "to": {
      "clear": {
        "weight": 0.9
      },
      "rain": {
        "weight": 0.1
      }
    }
  },
  "rain": {
    "from": {
      "clear": {
        "weight": 0.1
      },
      "rain": {
        "weight": 0.9
      }
    },
    "to": {
      "clear": {
        "weight": 0.1
      },
      "rain": {
        "weight": 0.9
      }
    }
  }
}


adjacency list instantiation

In [204]:
G = Markov.from_adj([[.9,.1],[.1,.9]],['aaaaaaaa','b'])
G.start('b')
print(G.pretty())

{
  "aaaaaaaa": {
    "from": {
      "aaaaaaaa": {
        "weight": 0.9
      },
      "b": {
        "weight": 0.1
      }
    },
    "to": {
      "aaaaaaaa": {
        "weight": 0.9
      },
      "b": {
        "weight": 0.1
      }
    }
  },
  "b": {
    "from": {
      "aaaaaaaa": {
        "weight": 0.1
      },
      "b": {
        "weight": 0.9
      }
    },
    "to": {
      "aaaaaaaa": {
        "weight": 0.1
      },
      "b": {
        "weight": 0.9
      }
    }
  }
}


In [198]:
for x in range(100):
    print(G.next())

aaaaaaaa
b
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
b
b
aaaaaaaa
b
b
b
b
aaaaaaaa
b
aaaaaaaa
b
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
b
aaaaaaaa
b
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
b
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
b
b
b
b
b
b
aaaaaaaa
b
b
b
b
b
aaaaaaaa
b
aaaaaaaa
b
b
b
b
b
b
b
b
b
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
b
aaaaaaaa
aaaaaaaa
b
b
aaaaaaaa
b
b
b
b
b
b
b
b
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
aaaaaaaa
b
b
b
b
